In [1]:
import os,openai
import pandas as pd
from typing import List, Tuple
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from sqlalchemy import create_engine
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI
import langchain
langchain.debug = False

import mysql.connector
from mysql.connector import pooling
from datetime import datetime  # Import datetime for timestamp handling
import logging
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set the OpenAI API key
openai_api_key = os.getenv("OPENAI_API_KEY")

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [31]:
# Get the variables from the environment
load_dotenv(dotenv_path=".env.dbdetails")
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = "localhost"  ## Docker dbhost is not connecting via jupyter  
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')

# Construct the MySQL URI
mysql_uri = f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

In [32]:
# Define MySQL URI
engine = create_engine(mysql_uri)

# Create SQLDatabase object from URI
db = SQLDatabase.from_uri(mysql_uri)
print(db.dialect)

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=1)
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools",return_intermediate_steps=False, return_direct=True)

mysql


In [19]:
import glob

def prepare_data():
    # Define data folder path
    data_folder = '../data/'

    # Get list of CSV files starting with "CRYPTOCURRENCY"
    files = glob.glob(data_folder + 'CRYPTOCURRENCY*.csv')

    # Initialize empty list to store DataFrames
    dfs = []
    # Load each CSV file into a DataFrame and append to list
    for file in files:
        df = pd.read_csv(file)
        dfs.append(df)

    # Concatenate all DataFrames into a single DataFrame
    combined_df = pd.concat(dfs, ignore_index=True)
    combined_df = combined_df.rename(columns={
    'cryptocurrency': 'Symbol'
    }).astype({
    'Date': 'datetime64[ns]',
    'Symbol': 'object'
    })
    
    
    # Remove commas from columns and then convert to float
    combined_df['Open'] = pd.to_numeric(combined_df['Open'].replace(',', '', regex=True), errors='coerce')
    combined_df['Close'] = pd.to_numeric(combined_df['Close'].replace(',', '', regex=True), errors='coerce')
    combined_df['Low'] = pd.to_numeric(combined_df['Low'].replace(',', '', regex=True), errors='coerce')
    combined_df['High'] = pd.to_numeric(combined_df['High'].replace(',', '', regex=True), errors='coerce')

    print("\nCombined DataFrame shape:", combined_df.shape)
    return combined_df

def create_db_table(combined_df):
    # Create table in MySQL database
    combined_df.to_sql("crypto_data", engine, index=False, if_exists="replace")
    
    # Test if table is created
    try:
        result = pd.read_sql_table("crypto_data", engine)
        print("\nTable 'crypto_data' exists and has shape:", result.shape)
    except Exception as e:
        print("\nError:", str(e))
        
combined_df = prepare_data()
create_db_table(combined_df)


Combined DataFrame shape: (732, 6)

Table 'crypto_data' exists and has shape: (732, 6)


### Pass user query and create sql query with llm

In [33]:
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many unique symbol are there in crypto_data"})
print(response)
db.run(response)

2024-10-12 11:17:30,004 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


SELECT COUNT(DISTINCT `Symbol`) AS unique_symbols FROM crypto_data;


'[(4,)]'

### Pass user query and create sql query with llm and pass the query result to llm to generate meaningful answer

In [21]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
#OutputParser that parses LLMResult into the top likely string.
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

def get_answer_from_sql(user_query):
    answer_prompt = PromptTemplate.from_template(
        """Given the following user question,corresponding SQL query, and SQL result, 
        answer the user question in upto 4 lines from the data provided in the SQL result.

        Question: {question}
        SQL Query: {query}
        SQL Result: {result}
        Answer: """
    )

    execute_query = QuerySQLDataBaseTool(db=db)
    write_query = create_sql_query_chain(llm, db)
    answer = answer_prompt | llm | StrOutputParser()

    chain = (
        RunnablePassthrough.assign(query=write_query).assign(
            result=itemgetter("query") | execute_query 
        )
        | answer_prompt
        | llm
        | StrOutputParser()
    )

    try:
        response = chain.invoke({"question": user_query})
        logging.info("Chain invoked successfully.")
        return response
    except Exception as e:
        logging.error(f"An error occurred: {e}")


user_query = "what is the lowest price for ethereum in april"
response = get_answer_from_sql(user_query)
print(response)


2024-10-12 11:06:07,111 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-12 11:06:07,787 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-12 11:06:07,793 - INFO - Chain invoked successfully.


The lowest price for Ethereum in April was $2864.54.


In [22]:
user_query = "which is cheap sol or eth overall"
response = get_answer_from_sql(user_query)
print(response)

2024-10-12 11:06:11,644 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-12 11:06:12,363 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-12 11:06:12,368 - INFO - Chain invoked successfully.


Sol is cheaper overall compared to ETH based on the provided data in the SQL result.


In [23]:
user_query = "Compare lowest price for dot with sol for april with date"
response = get_answer_from_sql(user_query)
print(response)

2024-10-12 11:06:14,902 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-12 11:06:15,708 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-12 11:06:15,715 - INFO - Chain invoked successfully.


The lowest price for DOT in April was 5.8121 on April 13th.


In [24]:
user_query = "which is cheap sol or dot overall"
get_answer_from_sql(user_query)

2024-10-12 11:06:18,050 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-12 11:06:18,827 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-12 11:06:18,832 - INFO - Chain invoked successfully.


'Based on the SQL result, DOT seems to be cheaper overall compared to SOL.'

In [25]:
user_query = "which is the most expensive overall"
get_answer_from_sql(user_query)

2024-10-12 11:06:20,655 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-12 11:06:21,579 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-12 11:06:21,581 - INFO - Chain invoked successfully.


'BTC has the highest overall high price of 72695.0.'